In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as thub
import bert
from tensorflow.keras import backend as K

import pandas as pd
import numpy as np

import re

import random

import os
from tqdm import tqdm

import model_utils

# maximum length of token sequences to input to bert model
max_seq_length = 128
# number of smaples to generate from the distributed database
n_samples = 5


/Users/chrisolen/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/chrisolen/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/chrisolen/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/chrisolen/anaconda3/lib/python3.7/site-packages/tensorflow/pyth

In [2]:
def tokenize_sample(context):
    
    """
    To be applied over Spark dataframe.
    Takes a string and converts it to token IDs via bert_tokenizer,
    adding the necessary beginning and end tokens

    Returns: Array of bert token ids for each row of Spark dataframe (requires udf)
    """
    
    tokenized = ["[CLS]"] + tokenizer.tokenize(context) + ["[SEP]"]
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    
    return ids

In [3]:
def generate_sample_df(sarcastic, non_sarcastic, ratio, n_samples):
    
    """
    Returns: Spark df of equal label distribution with text 
    tokenized. Each generated df is to be iterator over multiple 
    times during training
    """
    
    number = 0
    while number < n_samples:
        non_sarc_samp = non_sarcastic.sample(ratio) # making label dist equal
        
        # combine sampled non_sarcastic and whole sarcastic
        sample_df = sarcastic.union(non_sarc_samp)
        
        # tokenize context column via spark udf
        tokenize_sample_udf = F.udf(tokenize_sample, ArrayType(IntegerType()))
        sample_df = sample_df.withColumn("tokens", tokenize_sample_udf(sample_df.context))
        
        # drop context column
        sample_df = sample_df.drop("context")
        
        # yield one call at a time
        yield sample_df
        number += 1

In [16]:
class BertLayer(tf.keras.layers.Layer):
    
    def __init__(self,
        n_fine_tune_layers=2,
        output_type="sequence_output",
        bert_path="https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1",
        **kwargs):
        
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.output_type = output_type
        self.bert_path = bert_path
        
        if self.output_type not in ["sequence_output", "pooled_output"]:
            raise NameError("Undefined pooling type (must be either sequence_output or pooled_output, but is {self.output_type}")

        super(BertLayer, self).__init__(**kwargs)
        
    def layer_number(self, var):
    
        """
        Get the layer number corresponding to the 
        given variable
        """
        m = re.search(r'/layer_(\d+)/', var)
        
        if m:
            return int(m.group(1))
        else:
            return None


    def build(self, input_shape):

        """
        Creates the variables of the layer (optional, for subclass implementers).
 |      
 |      This is a method that implementers of subclasses of `Layer` or `Model`
 |      can override if they need a state-creation step in-between
 |      layer instantiation and layer call.
 |      
 |      This is typically used to create the weights of `Layer` subclasses.
 
        Called once from `__call__`, when we know the shapes of input and `dtype`.
        """
        
        self.bert_model = thub.KerasLayer(self.bert_path, self.trainable)

        # extract all trainable variables from the model
        trainable_vars = self.bert_model.trainable_variables
        
        if self.output_type == "pooled_output":
            
            # removing '/cls/' layers (there don't appear to be any) 
            trainable_vars = [var.name for var in trainable_vars if not "/cls/" in var.name]
            
        elif self.output_type == "sequence_output":
            
            # removing '/cls/' (there don't appear to be any) and '/pooler_transform/' layers 
            trainable_vars = [var.name for var in trainable_vars if not "/cls/" in var.name
                              and not "/pooler_transform" in var.name] 
            
        ### select how many layers to fine tune starting from top-most layer ###
        
        # outputs a list of either Nonetype or layer number
        layer_numbers = list(map(self.layer_number, trainable_vars))
        # returns the total number of layers in pre-trained model (note: layers are zero-indexed)
        n_total_layers = max(n for n in layer_numbers if n is not None) + 1 
        # finally, create list of just layers to be trained
        trainable_vars = [var for n, var in zip(layer_numbers, trainable_vars) if n is not None and n >= n_total_layers - self.n_fine_tune_layers]    
        
        # add variables NOT to be trained to _non_trainable_weights and 
        # remove them from _trainable_weights
        # note: underscore is necessary for accessing the writable object
        for var in self.bert_model.variables:

            if var.name not in trainable_vars and "Variable:0" not in var.name:
                
                    self.bert_model._non_trainable_weights.append(var)
                    self.bert_model._trainable_weights.remove(var)

        super(BertLayer, self).build(input_shape)
        
    def call(self, inputs): 
        
        """
        Called in `__call__` after making sure `build()` has been called
 |      once. Should actually perform the logic of applying the layer to the
 |      input tensors (which should be passed in as the first argument).
        """
        # takes in list of input tensors and casts them in Keras
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        
        #bert-for-tf2 returns (pooled_output,sequence_output) when called
        if self.output_type == "pooled_output":
            
            output = self.bert_model(inputs)[0]
                
        elif self.output_type == "sequence_output":
            
            output = self.bert_model(inputs)[1]
            
        return output
    
    def compute_output_shape(self, input_shape):
        
        return (input_shape[0], self.output_size)

       
        

In [6]:
class training(object):
    
    def __init__(self,
                max_seq_length=128,
                n_epochs=50,
                batch_size=13,
                patience=5,
                validation_split=0.1,
                checkpoint_dir=os.getcwd() + "/checkpoints",
                saved_model_dir=os.getcwd() + "/saved_models",
                pad_by_batch=False):
    
        self.max_seq_length = max_seq_length
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.patience = patience
        self.validation_split = validation_split
        self.checkpoint_dir = checkpoint_dir
        self.saved_model_dir = saved_model_dir
        self.pad_by_batch = pad_by_batch

    def build_model(self): 
    
        """
        Defines input shapes of bert input tensors,
        """
        input_word_ids = tf.keras.layers.Input(shape=(self.max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
        input_mask = tf.keras.layers.Input(shape=(self.max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
        segment_ids = tf.keras.layers.Input(shape=(self.max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
        bert_inputs = [input_word_ids, input_mask, segment_ids]
    
        bert_output = BertLayer()(bert_inputs)
        dense_out = tf.keras.layers.Dense(self.max_seq_length, activation='relu')(bert_output)
        dense_out = tf.keras.layers.Dropout(0.5)(dense_out)
        logits = tf.keras.layers.Dense(1, activation='sigmoid')(dense_out)
    
        model = tf.keras.models.Model(inputs=bert_inputs, outputs=logits)
        model.compile(loss='binary_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
                  metrics=['accuracy'])
    
        model.summary()
    
        return model
    
    def train_model(self, model, train_inputs, train_labels):
        
        """
        Initiates training process using self.build_model output as input.
        If self.pad_by_batch == False, input should be the entire epoch of 
        training inputs (including masks) and labels, as numpy arrays.
        """
    
        checkpoints = tf.keras.callbacks.ModelCheckpoint(self.checkpoint_dir, verbose=1, 
                                                     save_best_only=False,
                                                     save_weights_only=True, mode='auto', 
                                                     save_freq='epoch')
    
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=self.patience)
    
        if self.pad_by_batch == False:
            
            train_input_ids, train_input_masks, train_segment_ids = train_inputs
        
            model.fit([train_input_ids, train_input_masks, train_segment_ids], 
                  train_labels,
                  validation_split = self.validation_split,
                  epochs = self.n_epochs,
                  batch_size = self.batch_size,
                  callbacks = [checkpoints, early_stopping])
    
            model.save(self.saved_model_dir+'/my_model.h5')



In [6]:
# Initialize BERT model and tokenizer

%time bert_layer, tokenizer = model_utils.init_bert()

# Initialize Spark context

%time sc, spark = model_utils.init_spark()


CPU times: user 6.33 s, sys: 642 ms, total: 6.97 s
Wall time: 6.89 s
CPU times: user 33.6 ms, sys: 13.4 ms, total: 47 ms
Wall time: 9.98 s


In [7]:
# Read in sarcastic samples, non-sarcastic samples, and the ratio between the two

%time sarcastic, non_sarcastic, ratio = model_utils.load_data(spark, \
                                                              bucket_name="sarc-bucket-5", \
                                                              dataset="politics")

CPU times: user 10.9 ms, sys: 3.37 ms, total: 14.3 ms
Wall time: 27.7 s


In [8]:
# Initialize sample_df generator

%time sample_generator = generate_sample_df(sarcastic, non_sarcastic, ratio, n_samples)

# Output first smaple

%time sample_df = next(sample_generator)


CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 16.7 µs
CPU times: user 267 ms, sys: 7.2 ms, total: 274 ms
Wall time: 369 ms


In [17]:
# Initialize training class object and build Bert layer

t = training()
%time model = t.build_model()

   def call(self, inputs): 

       """
       Called in `__call__` after making sure `build()` has been called
|      once. Should actually perform the logic of applying the layer to the
|      input tensors (which should be passed in as the first argument).
       """
       # takes in list of input tensors and casts them in Keras
       inputs = [K.cast(x, dtype="int32") for x in inputs]

       #bert-for-tf2 returns (pooled_output,sequence_output) when called
       if self.output_type == "pooled_output":

           output = self.bert_model(inputs)[0]

       elif self.output_type == "sequence_output":

           output = self.bert_model(inputs)[1]

       return output

This may be caused by multiline strings or comments not indented at the same level as the code.


   def call(self, inputs): 

       """
       Called in `__call__` after making sure `build()` has been called
|      once. Should actually perform the logic of applying the layer to the
|      input tensors (which should be passed in as the first argument).
       """
       # takes in list of input tensors and casts them in Keras
       inputs = [K.cast(x, dtype="int32") for x in inputs]

       #bert-for-tf2 returns (pooled_output,sequence_output) when called
       if self.output_type == "pooled_output":

           output = self.bert_model(inputs)[0]

       elif self.output_type == "sequence_output":

           output = self.bert_model(inputs)[1]

       return output

This may be caused by multiline strings or comments not indented at the same level as the code.


   def call(self, inputs): 

       """
       Called in `__call__` after making sure `build()` has been called
|      once. Should actually perform the logic of applying the layer to the
|      input tensors (which should be passed in as the first argument).
       """
       # takes in list of input tensors and casts them in Keras
       inputs = [K.cast(x, dtype="int32") for x in inputs]

       #bert-for-tf2 returns (pooled_output,sequence_output) when called
       if self.output_type == "pooled_output":

           output = self.bert_model(inputs)[0]

       elif self.output_type == "sequence_output":

           output = self.bert_model(inputs)[1]

       return output

This may be caused by multiline strings or comments not indented at the same level as the code.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input

In [ ]:
# Produce padded tokens, input masks, and segment ids as nparrays

%time padded_tokens, train_labels = model_utils.pad(sample_df, pad_by_batch=False, t.batch_size)

%time input_mask = model_utils.input_mask(padded_tokens)

%time segment_id = model_utils.segment_id(padded_tokens)

train_inputs = [padded_tokens, input_mask, segment_id]


In [515]:
t.train_model(model, train_inputs, train_labels)

   def call(self, inputs): 

       """
       Called in `__call__` after making sure `build()` has been called
|      once. Should actually perform the logic of applying the layer to the
|      input tensors (which should be passed in as the first argument).
       """
       # takes in list of input tensors and casts them in Keras
       inputs = [K.cast(x, dtype="int32") for x in inputs]

       #bert-for-tf2 returns (pooled_output,sequence_output) when called
       if self.output_type == "pooled_output":

           output = self.bert_model(inputs)[0]

       elif self.output_type == "sequence_output":

           output = self.bert_model(inputs)[1]

       return output

This may be caused by multiline strings or comments not indented at the same level as the code.


   def call(self, inputs): 

       """
       Called in `__call__` after making sure `build()` has been called
|      once. Should actually perform the logic of applying the layer to the
|      input tensors (which should be passed in as the first argument).
       """
       # takes in list of input tensors and casts them in Keras
       inputs = [K.cast(x, dtype="int32") for x in inputs]

       #bert-for-tf2 returns (pooled_output,sequence_output) when called
       if self.output_type == "pooled_output":

           output = self.bert_model(inputs)[0]

       elif self.output_type == "sequence_output":

           output = self.bert_model(inputs)[1]

       return output

This may be caused by multiline strings or comments not indented at the same level as the code.


   def call(self, inputs): 

       """
       Called in `__call__` after making sure `build()` has been called
|      once. Should actually perform the logic of applying the layer to the
|      input tensors (which should be passed in as the first argument).
       """
       # takes in list of input tensors and casts them in Keras
       inputs = [K.cast(x, dtype="int32") for x in inputs]

       #bert-for-tf2 returns (pooled_output,sequence_output) when called
       if self.output_type == "pooled_output":

           output = self.bert_model(inputs)[0]

       elif self.output_type == "sequence_output":

           output = self.bert_model(inputs)[1]

       return output

This may be caused by multiline strings or comments not indented at the same level as the code.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp

In [7]:
sc.stop()

In [39]:
var_name = 'bert_layer_8/bert_model/pooler_transform/kernel:0'

In [40]:
trainable_vars = ['bert_layer_8/bert_model/encoder/layer_10/self_attention/query/kernel:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention/query/bias:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention/key/kernel:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention/key/bias:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention/value/kernel:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention/value/bias:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention_output/kernel:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention_output/bias:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention_layer_norm/gamma:0', 'bert_layer_8/bert_model/encoder/layer_10/self_attention_layer_norm/beta:0', 'bert_layer_8/bert_model/encoder/layer_10/intermediate/kernel:0', 'bert_layer_8/bert_model/encoder/layer_10/intermediate/bias:0', 'bert_layer_8/bert_model/encoder/layer_10/output/kernel:0', 'bert_layer_8/bert_model/encoder/layer_10/output/bias:0', 'bert_layer_8/bert_model/encoder/layer_10/output_layer_norm/gamma:0', 'bert_layer_8/bert_model/encoder/layer_10/output_layer_norm/beta:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention/query/kernel:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention/query/bias:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention/key/kernel:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention/key/bias:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention/value/kernel:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention/value/bias:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention_output/kernel:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention_output/bias:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention_layer_norm/gamma:0', 'bert_layer_8/bert_model/encoder/layer_11/self_attention_layer_norm/beta:0', 'bert_layer_8/bert_model/encoder/layer_11/intermediate/kernel:0', 'bert_layer_8/bert_model/encoder/layer_11/intermediate/bias:0', 'bert_layer_8/bert_model/encoder/layer_11/output/kernel:0', 'bert_layer_8/bert_model/encoder/layer_11/output/bias:0', 'bert_layer_8/bert_model/encoder/layer_11/output_layer_norm/gamma:0', 'bert_layer_8/bert_model/encoder/layer_11/output_layer_norm/beta:0']

In [42]:
(var_name not in trainable_vars) and ('Variable:0' not in var_name)

True

In [10]:
bert_layer, tokenizer = model_utils.init_bert()

In [15]:
[var.name for var in bert_layer.variables]

['bert_model/word_embeddings/embeddings:0',
 'bert_model/embedding_postprocessor/type_embeddings:0',
 'bert_model/embedding_postprocessor/position_embeddings:0',
 'bert_model/embedding_postprocessor/layer_norm/gamma:0',
 'bert_model/embedding_postprocessor/layer_norm/beta:0',
 'bert_model/encoder/layer_0/self_attention/query/kernel:0',
 'bert_model/encoder/layer_0/self_attention/query/bias:0',
 'bert_model/encoder/layer_0/self_attention/key/kernel:0',
 'bert_model/encoder/layer_0/self_attention/key/bias:0',
 'bert_model/encoder/layer_0/self_attention/value/kernel:0',
 'bert_model/encoder/layer_0/self_attention/value/bias:0',
 'bert_model/encoder/layer_0/self_attention_output/kernel:0',
 'bert_model/encoder/layer_0/self_attention_output/bias:0',
 'bert_model/encoder/layer_0/self_attention_layer_norm/gamma:0',
 'bert_model/encoder/layer_0/self_attention_layer_norm/beta:0',
 'bert_model/encoder/layer_0/intermediate/kernel:0',
 'bert_model/encoder/layer_0/intermediate/bias:0',
 'bert_model